In [1]:
import os

# Modify the following values accordingly
os.environ["AWS_ACCESS_KEY_ID"] = "<your_aws_access_key_id>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<your_aws_secret_access_key>"
os.environ["AWS_REGION_NAME"] = "<your_aws_region_name>"


question = """What's the longest river in the world?"""

response1 = """
The longest river in the world is the Nile River, located in northeastern Africa. 
It stretches for approximately 6,853 kilometers (4,258 miles) from its sources in Burundi, Rwanda, and Tanzania to its delta on the Mediterranean Sea in Egypt.
"""

response2 = """
The longest river in the world is the Nile River. 
It flows northward through northeastern Africa for approximately 6,650 kilometers (4,132 miles) from its most distant source in the African Great Lakes region to the Mediterranean Sea.
"""

reference = """
The Nile is a major north-flowing river in northeastern Africa. 
It flows into the Mediterranean Sea. The Nile is the longest river in Africa and has historically been considered the longest river in the world, though this has been contested by research suggesting that the Amazon River is slightly longer. 
Of the world's major rivers, the Nile is one of the smallest, as measured by annual flow in cubic metres of water. 
About 6,650 km (4,130 mi) long, its drainage basin covers eleven countries: the Democratic Republic of the Congo, Tanzania, Burundi, Rwanda, Uganda, Kenya, Ethiopia, Eritrea, South Sudan, Sudan, and Egypt. 
In particular, the Nile is the primary water source of Egypt, Sudan and South Sudan. 
Additionally, the Nile is an important economic river, supporting agriculture and fishing.
"""

In [2]:
from refchecker import LLMExtractor

# claim extraction
extractor = LLMExtractor(
    claim_format='triplet', 
    model='bedrock/anthropic.claude-3-sonnet-20240229-v1:0',
    batch_size=8
)

# each element in claims is an instance of Claim
extraction_results = extractor.extract(
    batch_responses=[response1, response2],
    max_new_tokens=1000
)
for i, res in enumerate(extraction_results):
    print(f'Claims in Response {i+1}:')
    for claim in res.claims:
        print(claim.content)
    print('----')

100%|██████████| 1/1 [00:02<00:00,  2.80s/it]

Claims in Response 1:
['The longest river in the world', 'is', 'the Nile River']
['The Nile River', 'located in', 'northeastern Africa']
['The Nile River', 'stretches for', 'approximately 6,853 kilometers (4,258 miles)']
['The Nile River', 'has sources in', 'Burundi']
['The Nile River', 'has sources in', 'Rwanda']
['The Nile River', 'has sources in', 'Tanzania']
['The Nile River', 'has delta on', 'the Mediterranean Sea']
['The Nile River delta', 'located in', 'Egypt']
----
Claims in Response 2:
['The longest river in the world', 'is', 'the Nile River']
['the Nile River', 'flows', 'northward']
['the Nile River', 'flows through', 'northeastern Africa']
['the Nile River', 'has length of approximately', '6,650 kilometers (4,132 miles)']
['the Nile River', 'originates from', 'its most distant source in the African Great Lakes region']
['the Nile River', 'ends at', 'the Mediterranean Sea']
['the African Great Lakes region', 'is source of', 'the Nile River']
['the Mediterranean Sea', 'is dest

In [3]:
from refchecker import LLMChecker

checker = LLMChecker(model='bedrock/anthropic.claude-3-sonnet-20240229-v1:0')

batch_claims = []
for res in extraction_results:
    batch_claims.append([claim.content for claim in res.claims])

batch_reference = [reference] * len(batch_claims)

checking_results = checker.check(
    batch_claims=batch_claims,
    batch_references=batch_reference,
    max_reference_segment_length=0
)

for i, (extract_res, check_res) in enumerate(zip(extraction_results, checking_results)):
    print(f'Checking results for Response {i+1}:')
    for claim, pred_label in zip(extract_res.claims, check_res):
        print(f'{claim.content} --> {pred_label}')
    print('---')

100%|██████████| 1/1 [00:01<00:00,  1.55s/it]

Checking results for Response 1:
['The longest river in the world', 'is', 'the Nile River'] --> Contradiction
['The Nile River', 'located in', 'northeastern Africa'] --> Entailment
['The Nile River', 'stretches for', 'approximately 6,853 kilometers (4,258 miles)'] --> Entailment
['The Nile River', 'has sources in', 'Burundi'] --> Entailment
['The Nile River', 'has sources in', 'Rwanda'] --> Entailment
['The Nile River', 'has sources in', 'Tanzania'] --> Entailment
['The Nile River', 'has delta on', 'the Mediterranean Sea'] --> Entailment
['The Nile River delta', 'located in', 'Egypt'] --> Entailment
---
Checking results for Response 2:
['The longest river in the world', 'is', 'the Nile River'] --> Contradiction
['the Nile River', 'flows', 'northward'] --> Entailment
['the Nile River', 'flows through', 'northeastern Africa'] --> Entailment
['the Nile River', 'has length of approximately', '6,650 kilometers (4,132 miles)'] --> Entailment
['the Nile River', 'originates from', 'its most di